In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import shutil
shutil.unpack_archive("/content/drive/MyDrive/neww.zip", "dinnerTrayBreakfastTray/")

In [23]:

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
from keras.callbacks import ModelCheckpoint
# Import the Desired Version of EfficientNet
from tensorflow.keras.applications import EfficientNetB0

# Variables
NUM_CLASSES = 2

train_path = "/content/dinnerTrayBreakfastTray/new/training"
valid_path = "/content/dinnerTrayBreakfastTray/new/valid"
test_path = "/content/dinnerTrayBreakfastTray/new/test"

epochs = 50

model_save_location = "/content/newsave"


img_augmentation = Sequential(
    [
        preprocessing.RandomRotation(factor=0.15),
        preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        preprocessing.RandomFlip(),
        preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

def build_model(NUM_CLASSES):
    inputs = layers.Input(shape=(224, 224, 3))
    x = img_augmentation(inputs)

    #Using the imported version of EfficientNet
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

def test_model(model,test_batches):
    #Testing the Model
    test_labels = test_batches.classes
    print("Test Labels",test_labels)
    print(test_batches.class_indices)

    predictions = model.predict(test_batches,steps=len(test_batches),verbose=0)

    acc = 0
    for i in range(len(test_labels)):
        actual_class = test_labels[i]
        if predictions[i][actual_class] > 0.5 :
            acc += 1
    print("Accuarcy:",(acc/len(test_labels))*100,"%")

if __name__ == "__main__":
    model = build_model(NUM_CLASSES)
    unfreeze_model(model)

    train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
        directory=train_path, target_size=(224,224), batch_size=10)
    valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
        directory=valid_path, target_size=(224,224), batch_size=10)
    test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
        directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

    checkpoint = ModelCheckpoint(filepath='mymodel.h5',
                               verbose=2,save_best_only=True)
    callbacks = [checkpoint]
    start = datetime.now()



    _ = model.fit(train_batches,
                  epochs=epochs,
                  validation_data=valid_batches, callbacks=callbacks,
                  verbose=1)

    duration = datetime.now() - start
    print("Training completed in time: ", duration)
    #Testing the Model
    test_model(model,test_batches)

    # Save the tensorflow Model
    #model.save(model_save_location)

Found 389 images belonging to 2 classes.
Found 29 images belonging to 2 classes.
Found 17 images belonging to 2 classes.
Epoch 1/50


39/39 [==============================] - ETA: 0s - loss: 0.3095 - accuracy: 0.8715
Epoch 1: val_loss improved from inf to 0.19757, saving model to mymodel.h5
39/39 [==============================] - 26s 387ms/step - loss: 0.3095 - accuracy: 0.8715 - val_loss: 0.1976 - val_accuracy: 0.9655
Epoch 2/50
39/39 [==============================] - ETA: 0s - loss: 0.1422 - accuracy: 0.9486
Epoch 2: val_loss improved from 0.19757 to 0.09717, saving model to mymodel.h5
39/39 [==============================] - 13s 320ms/step - loss: 0.1422 - accuracy: 0.9486 - val_loss: 0.0972 - val_accuracy: 1.0000
Epoch 3/50
39/39 [==============================] - ETA: 0s - loss: 0.0929 - accuracy: 0.9717
Epoch 3: val_loss improved from 0.09717 to 0.05935, saving model to mymodel.h5
39/39 [==============================] - 12s 313ms/step - loss: 0.0929 - accuracy: 0.9717 - val_loss: 0.0593 - val_accuracy: 1.0000
Epoch 4/50
39/39 [==============================] - ETA: 0s - loss: 0.0648 - accuracy: 0.9769
Epoch 

In [24]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.data import AUTOTUNE
from tensorflow.keras.models import load_model
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD

#--SK
from sklearn.metrics import classification_report

#--Others
import imutils
from imutils import paths
import random
import cv2
import os
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt

In [25]:
test_paths = list(paths.list_images(test_path))
testlabels = [int(p.split(os.path.sep)[-2]) for p in test_paths]
#turn them into categorical values so we can count them (one hot encoding)
testlabels = to_categorical(testlabels)

In [26]:
Y_pred = model.predict(test_batches)
y_pred = np.argmax(Y_pred, axis=1)

2/2 [==============================] - 1s 195ms/step


In [27]:
print('Classification Report')
print(classification_report(test_batches.classes, y_pred, target_names=["0","1"]))

Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         9

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17

